### SQL(SQLalchemy)

`SQLite` 사용

pip install -qU langchain-community SQLAlchemy langchain-openai

In [1]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHisory객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달.
# session_id은 sqldb안에 column, connection에서 sqlite은 파일명. 내용은 한글이라 깨져보일 수 있다.
chat_message_history = SQLChatMessageHistory(
    session_id = "sql_history", connection="sqlite:///sqlite.db" 
)

In [3]:
# 사용자 메세지 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내이름은 재호야. 나는 랭체인과 slm개발자를 희망하고 있어. 앞으로 잘 부탁해. "
)
# AI 메세지 추가
chat_message_history.add_ai_message("안녕 재호! 만나서 반가워. 나도 잘 부탁해")

In [5]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHisory객체를 생성하고 세션 ID와 데이터베이스 연결 문자열을 전달.
# session_id은 sqldb안에 column, connection에서 sqlite은 파일명. 내용은 한글이라 깨져보일 수 있다.
chat_message_history =SQLChatMessageHistory(
    session_id = "sql_history2", connection="sqlite:///sqlite.db" 
)

In [6]:
# 사용자 메세지 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내이름은 진원이야. 나는 성악과, 음악치료를 전공했어 그리고 재호랑 결혼할 사이야! 앞으로 잘 부탁해."
)
# AI 메세지 추가
chat_message_history.add_ai_message("안녕 진원! 만나서 반가워. 나도 잘 부탁해")

저장된 대화내용 확인`chat_message_history.messages`

# Chain에 적용

In [7]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful assistant'),
        MessagesPlaceholder(variable_name="chat_history"),
        ('human', '{question}')
    ]
)
chain = prompt | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()

`splite.db`.에서 대화내용을 가져오는 함수를 만든다.

- `user_id`는 사용자 ID,
- `conversation_id`는 대화ID

In [6]:
# conversation_id은 thread id라 생각하면 된다.
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name = user_id,
        session_id = conversation_id,
        connection="sqlite:///sqlite.db"
    )

In [9]:
 # 함수의 파라미터값과, config의 id가 일치해야한다.
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Uniqte identifier for a conversation.",
        default="",
        is_shared=True,
    )
]

In [10]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history, # 대화 기록을 저장하는 함수
    input_messages_key="question", # 입력 메세지의 키를 'question'으로 저장. prompt의 {question}과 매핑이 되는것.
    history_messages_key="chat_history", # 대화 내용 기록을 chat_history로 저장
    history_factory_config=config_fields # 대화 기록 조회시 참고할 파라미터를 설정. config_field를 매밍해준다. 
    # get_chat_history함수는 원래를 인자를 session_id로 하나만 받기로 되어있는데, 
    # 인자의 이름을 변경하거나, 개수를 늘려주고 싶으면 ConfigurableFieldSpec를 정의하고, history_factory_config에 넣어줘야한다.
)

In [13]:
# 세션 ID를 구성하는 곳
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

질문에 이름을 물어보는 질문을 해보겠습니다. 이전에 저장한 대화가 있다면, 올바르게 답할 것입니다.

- `chain_with_history` 객체의 `invok`e 메서드를 호출하여 질문에 대한 답변을 생성합니다.
- `invoke` 메서드에는 질문 딕셔너리와 `config` 설정이 전달됩니다

In [14]:
chain_with_history.invoke({"question": "안녕 반가워 내이름은 재호야"}, config)

'안녕하세요, 재호님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [15]:
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'재호님이라고 하셨죠! 맞나요?'

In [16]:
config2 = {"configurable": {"user_id": "user2", "conversation_id": "conversation2"}}

In [17]:
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config2)

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 좋겠습니다!'